# MENTORÍA 06
# **Cambio climático y ML: cómo mitigar las emisiones de CO2 mediante la reducción del consumo energético en construcciones edilicias**

**TP N°2: Análisis exploratorio y curación**

Fecha de entrega: **29 de julio**

## Consignas generales

En este segundo TP deberemos asegurarnos de que nuestro dataset esté listo para introducirlo a un modelo.
Al igual que en el TP anterior, les propongo la elaboración de un informe donde se aclare o se detalle las decisiones tomadas.
Podemos tener más de un set de datos (o escenarios), de forma tal de poder analizar combinaciones diferentes de features que creamos puedan tener incidencia en nuestras predicciones. Les propongo contar con al menos dos escenarios (pueden evaluar más de dos, por supuesto):

    - Escenario base: datos sin escalar y con todas las features.
    - Escenario 2: datos escalados y con las mejoras que hayan realizado.
    
De esta manera, podremos comparar qué tan buenas fueron nuestras elecciones, al contrastar las métricas que obtendremos con el peor de los modelos que podríamos haber usado.

---

## Pasos a seguir para evitar el train-test data leaking

1- Renombrar columnas o transformar unidades de medición.

2- Chequear datos repetidos.

3- Chequear "missingness" o valores faltantes:

Verificar que las columnas numéricas correspondan a un tipo de dato numérico. Muchas veces, los valores faltantes se llenan con algún string como "NA", '.', o "-". En ese caso, un dato numérico (float o int) tendrá el tipo object.
    
        1- df.info() --> chequeo tipo de datos
        2- df.coldeinteres.unique() --> si veo una columna misteriosa, analizo los valores que contiene
        3- df.describe --> Si con un describe (o un value_counts) identifican valores == 0 en features que no deberían tener 0, entonces, ahí hay otro dato falante incorporado como 0.

Si encuentran algo así, deben reemplazar esos valores con "missing value" o NaN:

        1- df = pd.read_csv('data.csv', na_values='-')
        2- df.replace('-', np.nan)
        2- df.coldeinteres[df.coldeinteres == 0] = np.nan

Si deciden eliminar alguna columna por presentar muchos valores faltantes, pueden hacerlo en esta etapa.

Pero, si deciden imputar, deberán hacerlo más adelante.
    
----------------------------------------------------------------------------------------------------------------------------------

Los puntos anteriores los pueden realizar con la totalidad del dataset. Sin embargo, previo a trabajar con las variables numéricas o cuantitativas que requieran escalado, chequeo de outliers, o evaluación de valores faltantes para imputar, debemos **DIVIDIR EL DATA SET EN TRAIN Y TEST**. De lo contrario, estaríamos comentiendo lo que se conoce como **contaminación train-test (un tipo de data leaking).**

-----------------------------------------------------------------------------------------------------------------------------------

4- Separar la variable target y dividir el set de datos en train y test.

        X = df.drop(['target'],axis=1)
        y = df['target']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)        
        
5- Luego de identificar las variables categóricas o cualitativas, evaluar su distribución, reasignarlas (o no), encodearlas por separado. Pueden usar getdummies, OneHotEncoding o OrdinalEncoding, dependiendo el objetivo.
Durante el proceso, recordar dropear una de las categorías para evitar la multicolinealidad.
    
        X_train = pd.get_dummies(X_train, columns=["catcol1", "catcol2"], drop_first=True)
        X_test = pd.get_dummies(X_test, columns=["catcol1", "catcol2"], drop_first=True)
        
6- Escalar variables numéricas con el método .fit_transform en el set de entrenamiento, pero sólo con el método .transform en el set de testeo. Esto se hace para utilizar la media y el desvío calculados (si usamos StandardScaler, por ejemplo) únicamente con los datos de entrenamiento. Recuerden que los datos de test son datos que simulan provenir de la realidad, posterior al entrenamiento del modelo, y su función radica en testear que tan bien generaliza mi modelo con datos nuevos (ya que para eso estamos haciendo y entrenando nuestros modelos: para que predigan lo mejor posible sobre nuevos valores de features que nunca vio). Ejemplos:

        scaler = StandardScaler()
        scaler.fit_transform(X_train)
        scaler.transform(X_test)
        
        scaler = MinMaxScaler()
        num_cols = ["numcol1","numcol2","numcol3"]
        X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
        X_test[num_cols] = scaler.transform(X_test[num_cols])

7- El proceso de imputar es similar al de escalado:

        # Imputación
        my_imputer = SimpleImputer()
        imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
        imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

        # Como la imputación elimina los nombres de las columnas, hay que agregarlos de nuevo
        imputed_X_train.columns = X_train.columns
        imputed_X_test.columns = X_test.columns
        
-------------------------------------------------------------------------------------------------------------------------------------

En cuanto al análisis de los outliers, va a depender si los necesitamos para corregir valores atípicos (temperaturas de 1000°C, por ejemplo) o si vamos a depender del análisis de la distribución de los datos y, por ende, de la mediana. En el primer caso, podemos usar todo el dataset. Pero en el segundo, deberíamos trabajar el set de train y le de test por separado.


Fuentes:

https://jahazielponce.com/kaggle-30-dias-ml-dias-12-14/

https://towardsdatascience.com/the-dreaded-antagonist-data-leakage-in-machine-learning-5f08679852cc

---

# **ESCENARIOS**

## ESCENEARIO BASE





DESCRIPCION:
* Datos sin escalar.
* Todas las features.
* Imputacion de valores faltantes mediante _'SimpleImputer'_
* Encoding de variables categoricas mediante _'get_dummies'_


---

## ESCENEARIO 1





DESCRIPCION:
* Datos escalados con _'StandardScaler'_.
* Todas las features.
* Imputacion de valores faltantes mediante _'SimpleImputer'_
* Encoding de variables categoricas mediante _'get_dummies'_


---

## ESCENEARIO 2

**Escenerio conservador** donde se tienen en cuenta no solo todas las variables
que se consideran relevantes sino tambien algunas adicionales, no se eliminan registros, se imputan datos faltantes.

DESCRIPCION:

* Datos escalados.
* Imputcion de valores faltantes:
  * Se hace la imputacion en las variables 'año_construccion' (por su bajo porcentaje de valor faltantes, **2,4%**) y 'calificacion_energystar' (por su importancia para predecir la variable target, **-0,51 de correlacion**).
* Variables categoricas:
  * Eliminacion de 'clase_edificio', la informacion esta contemplada en 'tipo_instalacion'.
  * Codificadas con metodo _'get_dummies'_
* Variables de temperatura por mes:
  * Eliminacion de todas las 'nombreMes_temp_promedio', se conservan solo los minimos y maximos de cada mes, con esta informacion ya se esta contemplando el promedio.
* Variables meteorologicas:
  * Eliminacion de las 3 variables asociadas al viento ('direccion_velocidad_viento_maxima', 'direccion_velocidad_viento_pico' y 'velocidad_viento_maxima') y la de niebla ('dias_con_niebla'), por el gran procentaje de valores faltantes, **>50%**.
  *

* Dudas:
  * En la variable 'año_construccion' hay 6 datos con el valor 0 (cero): ¿como afecta esto al modelo?.

      
     
              




---

## ESCENEARIO 3

**Escenerio arriesgado** donde solo se tienen en cuenta las variables que se consideran relevantes, se eliminan registros que se consideran que pueden afectar al modelo, no se imputan datos faltantes para no agregar informacion estimada al data set.

DESCRIPCION:

* Datos escalados.
* Valores faltantes:
  * En aquellas variables que contengan datos faltantes, se elimina directamente el registro, para no introducir error en el set de datos.
* Variables categoricas:
  * Eliminacion de 'clase_edificio', la informacion esta contemplada en 'tipo_instalacion'.
* Variables de temperatura:
  * Solo se considera 'temperatura_promedio'.
* Variables meteorologicas:
  * Eliminacion de las 3 variables asociadas al viento ('direccion_velocidad_viento_maxima', 'direccion_velocidad_viento_pico' y 'velocidad_viento_maxima') y la de niebla ('dias_con_niebla'), por el gran procentaje de valores faltantes, **>50%**.
  




---

## ESCENEARIO 4

DESCRIPCION:

+ Eliminación de las variables climaticas
 + Variables relacionadas a viento y niebla
 + Variables de rango de temperatura, quitando minimo y maximo para solo conservar promedio por mes
+ Se descarta la variable "tipo de instalación" preeviendo que su informacion este correctamtente representada en la variables "clase de edificio"
+ La implementacion de un pipeline que impute los datos faltantes con "_IterativeImputer_" con un máximo de 100 iteraciones y luego escale los datos con "_StandardScaler_".




---

## ESCENEARIO 5

DESCRIPCION:

+ Eliminación de las variables climaticas
 + Variables relacionadas a viento y niebla
 + Variables de temperatura promedio, valor minimo y valor maximo
+ Creación de una nueva variable climatica donde se promedian los meses agrupados por estacion.  
+ Nuevamente se descarta la variable "tipo de instalación" preeviendo que su informacion este correctamtente representada en la variables "clase de edificio"
+ La implementacion de un pipeline que impute los datos faltantes con "SimpleImputer" utilizando la **mediana** y luego escale los datos con "_StandardScaler_".



---

# CURACION DE DATOS

### 1) PASOS EN COMUN PARA TODOS LOS ESCENARIOS

In [ ]:
# Instalacion
!pip install pandas numpy seaborn matplotlib missingno

In [ ]:
# Importacion
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
# Importacion del data set
df = pd.read_csv("data.csv")

#### 1.1) Renombrar columnas y transformar unidades de medición

In [ ]:
# Renombrar columnas
df=df.rename(columns={
    'Year_Factor': 'factor_año',
    'State_Factor': 'factor_estado',
    'building_class': 'clase_edificio',
    'facility_type': 'tipo_instalacion',
    'floor_area': 'area_edificio',
    'year_built': 'año_construccion',
    'energy_star_rating': 'calificacion_energystar',
    'ELEVATION': 'elevacion',
    'january_min_temp': 'enero_temp_min',
    'january_avg_temp': 'enero_temp_promedio',
    'january_max_temp': 'enero_temp_max',
    'february_min_temp': 'febrero_temp_min',
    'february_avg_temp': 'febrero_temp_promedio',
    'february_max_temp': 'febrero_temp_max',
    'march_min_temp': 'marzo_temp_min',
    'march_avg_temp': 'marzo_temp_promedio',
    'march_max_temp': 'marzo_temp_max',
    'april_min_temp': 'abril_temp_min',
    'april_avg_temp': 'abril_temp_promedio',
    'april_max_temp': 'abril_temp_max',
    'may_min_temp': 'mayo_temp_min',
    'may_avg_temp': 'mayo_temp_promedio',
    'may_max_temp': 'mayo_temp_max',
    'june_min_temp': 'junio_temp_min',
    'june_avg_temp': 'junio_temp_promedio',
    'june_max_temp': 'junio_temp_max',
    'july_min_temp': 'julio_temp_min',
    'july_avg_temp': 'julio_temp_promedio',
    'july_max_temp': 'julio_temp_max',
    'august_min_temp': 'agosto_temp_min',
    'august_avg_temp': 'agosto_temp_promedio',
    'august_max_temp': 'agosto_temp_max',
    'september_min_temp': 'septiembre_temp_min',
    'september_avg_temp': 'septiembre_temp_promedio',
    'september_max_temp': 'septiembre_temp_max',
    'october_min_temp': 'octubre_temp_min',
    'october_avg_temp': 'octubre_temp_promedio',
    'october_max_temp': 'octubre_temp_max',
    'november_min_temp': 'noviembre_temp_min',
    'november_avg_temp': 'noviembre_temp_promedio',
    'november_max_temp': 'noviembre_temp_max',
    'december_min_temp': 'diciembre_temp_min',
    'december_avg_temp': 'diciembre_temp_promedio',
    'december_max_temp': 'diciembre_temp_max',
    'cooling_degree_days': 'grados_dia_enfriamiento',
    'heating_degree_days': 'grados_dia_calefaccion',
    'precipitation_inches': 'precipitacion_mm',
    'snowfall_inches': 'nevada_mm',
    'snowdepth_inches': 'profundidad_nieve_mm',
    'avg_temperature': 'temperatura_promedio',
    'days_below_30F':'dias_menos_1C',
    'days_below_20F': 'dias_menos_6C',
    'days_below_10F': 'dias_menos_12C',
    'days_below_0F': 'dias_menos_18C',
    'days_above_80F': 'dias_mas_27C',
    'days_above_90F': 'dias_mas_32C',
    'days_above_100F': 'dias_mas_38C',
    'days_above_110F': 'dias_mas_43C',
    'direction_max_wind_speed': 'direccion_velocidad_viento_maxima',
    'direction_peak_wind_speed': 'direccion_velocidad_viento_pico',
    'max_wind_speed': 'velocidad_viento_maxima',
    'days_with_fog': 'dias_con_niebla',
    'site_eui': 'consumo',
})

In [ ]:
# Cambio de unidades

  # consumo -> kBtu/ft² to kWh/m²
df['consumo']= df['consumo'] * 3.1546
df['consumo'] = df['consumo'].round(2)

  # temperatura -> °F to °C
temp_cols = [col for col in df.columns if 'temp' in col]
df_temp = df[temp_cols]
df_temp_C=(df_temp-32)* (5/9)
df_temp_C = df_temp_C.round(2)
df[temp_cols] = df_temp_C

  # precipitaciones -> inches to mm
precip_cols = [col for col in df.columns if 'inches' in col]
df_precip = df[precip_cols]
df_precip_mm = df_precip * 25.4
df_precip_mm = df_precip_mm.round(2)
df[precip_cols] = df_precip_mm

  # areas -> ft² to m²
df['area_edificio']= df['area_edificio'] * 0.092903
df['area_edificio'] = df['area_edificio'].round(2)

  # velocidad -> mph to km/h
df['velocidad_viento_maxima']= df['velocidad_viento_maxima'] * 1.60934
df['velocidad_viento_maxima'] = df['velocidad_viento_maxima'].round(2)

#### 1.2) Chequear datos repetidos

In [ ]:
print("Filas duplicadas:")
print(df.duplicated().sum())

print("\nFilas duplicadas (mostrando las duplicadas):")
print(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)))


Filas duplicadas:
0

Filas duplicadas (mostrando las duplicadas):
Empty DataFrame
Columns: [factor_año, factor_estado, clase_edificio, tipo_instalacion, area_edificio, año_construccion, calificacion_energystar, elevacion, enero_temp_min, enero_temp_promedio, enero_temp_max, febrero_temp_min, febrero_temp_promedio, febrero_temp_max, marzo_temp_min, marzo_temp_promedio, marzo_temp_max, abril_temp_min, abril_temp_promedio, abril_temp_max, mayo_temp_min, mayo_temp_promedio, mayo_temp_max, junio_temp_min, junio_temp_promedio, junio_temp_max, julio_temp_min, julio_temp_promedio, julio_temp_max, agosto_temp_min, agosto_temp_promedio, agosto_temp_max, septiembre_temp_min, septiembre_temp_promedio, septiembre_temp_max, octubre_temp_min, octubre_temp_promedio, octubre_temp_max, noviembre_temp_min, noviembre_temp_promedio, noviembre_temp_max, diciembre_temp_min, diciembre_temp_promedio, diciembre_temp_max, grados_dia_enfriamiento, grados_dia_calefaccion, precipitacion_mm, nevada_mm, profundidad_n

#### 1.3) Chequear "missingness" o valores faltantes




In [ ]:
# Identificar columnas con tipo 'object'
object_cols = df.select_dtypes(include='object').columns

# Verificar si hay valores como "NA", ".", o "-" en columnas 'object'
missing_markers = ["NA", "\.", "-"]
for col in object_cols:
  for marker in missing_markers:
    if df[col].astype(str).str.contains(marker).any():
      print(f"La columna '{col}' de tipo 'object' contiene el valor '{marker}'")

No se observan caracteres especiales en las variables categoricas

### 2) PASOS PARA ESCENARIO BASE

#### 2.1) Separar la variable target y dividir el set de datos en train y test.

In [ ]:
# Copia del data set para el escenario base
df_escBase = df.copy()

# Separacion de la variable target
X_escBase = df_escBase.drop('consumo', axis=1)
y_escBase = df_escBase['consumo']

# Dividir el set de datos de entrenamiento y testeo (70-30)
X_train_escBase, X_test_escBase, y_train_escBase, y_test_escBase = train_test_split(X_escBase, y_escBase, test_size=0.3, random_state=42)

print("Separacion en set de train y test completo usando train_tes_split.")
print(f"X_train_escBase shape: {X_train_escBase.shape}")
print(f"X_test_escBase shape: {X_test_escBase.shape}")
print(f"y_train_escBase shape: {y_train_escBase.shape}")
print(f"y_test_escBase shape: {y_test_escBase.shape}")


Separacion en set de train y test completo usando train_tes_split.
X_train_escBase shape: (53029, 63)
X_test_escBase shape: (22728, 63)
y_train_escBase shape: (53029,)
y_test_escBase shape: (22728,)


#### 2.2) Encoding

 Luego de identificar las variables categóricas o cualitativas, evaluar su distribución, reasignarlas (o no), encodearlas por separado. Pueden usar getdummies, OneHotEncoding o OrdinalEncoding, dependiendo el objetivo. Durante el proceso, recordar dropear una de las categorías para evitar la multicolinealidad.

In [ ]:
# Identificar las columnas categoricas y numericas para X_train y X_test
categorical_cols = X_train_escBase.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train_escBase.select_dtypes(include=np.number).columns

# Encoding mediante get_dummies
X_train_escBase_encoded = pd.get_dummies(X_train_escBase, columns=categorical_cols, drop_first=True, dtype=int)
X_test_escBase_encoded = pd.get_dummies(X_test_escBase, columns=categorical_cols, drop_first=True, dtype=int)

print("Encoding completo usando get_dummies.")
print(f"X_train_escBase_encoded shape: {X_train_escBase_encoded.shape}")
print(f"X_test_escBase_encoded shape: {X_test_escBase_encoded.shape}")

Encoding completo usando get_dummies.
X_train_escBase_encoded shape: (53029, 126)
X_test_escBase_encoded shape: (22728, 126)


#### 2.3) Imputacion

In [ ]:
print("Valores faltantes en X_train (ANTES de la imputacion)")
print(X_train_escBase_encoded.isnull().sum().sort_values(ascending=False).head(10))

print("\nValores faltantes en X_test (ANTES de la imputacion)")
print(X_test_escBase_encoded.isnull().sum().sort_values(ascending=False).head(10))


# Inicializacion de SimpleImputer (mean)
imputer_escBase = SimpleImputer(strategy='mean')

# Columnas numericas despues del encoding, se elimina 'id'
numerical_cols_after_encoding = X_train_escBase_encoded.select_dtypes(include=np.number).columns.tolist()
if 'id' in numerical_cols_after_encoding:
    numerical_cols_after_encoding.remove('id')


# Imputar X_train
X_train_escBase_encoded[numerical_cols_after_encoding] = imputer_escBase.fit_transform(X_train_escBase_encoded[numerical_cols_after_encoding])

# Imputar X_test
X_test_escBase_encoded[numerical_cols_after_encoding] = imputer_escBase.transform(X_test_escBase_encoded[numerical_cols_after_encoding])


print("\nValores faltantes en X_train (DESPUES de la imputacion)")
print(X_train_escBase_encoded.isnull().sum().sort_values(ascending=False).head(10))

print("\nValores faltantes en X_test (DESPUES de la imputacion)")
print(X_test_escBase_encoded.isnull().sum().sort_values(ascending=False).head(10))



Valores faltantes en X_train (ANTES de la imputacion)
dias_con_niebla                      32023
direccion_velocidad_viento_pico      29380
direccion_velocidad_viento_maxima    28847
velocidad_viento_maxima              28847
calificacion_energystar              18832
año_construccion                      1280
enero_temp_promedio                      0
enero_temp_max                           0
febrero_temp_min                         0
febrero_temp_promedio                    0
dtype: int64

Valores faltantes en X_test (ANTES de la imputacion)
dias_con_niebla                      13773
direccion_velocidad_viento_pico      12431
direccion_velocidad_viento_maxima    12235
velocidad_viento_maxima              12235
calificacion_energystar               7877
año_construccion                       557
enero_temp_promedio                      0
enero_temp_max                           0
febrero_temp_min                         0
febrero_temp_promedio                    0
dtype: int64

Valor

In [ ]:
# Set de datos finales para Escenario Base
X_train_final_escBase = X_train_escBase_encoded
X_test_final_escBase = X_test_escBase_encoded
y_train_final_escBase = y_train_escBase
y_test_final_escBase = y_test_escBase

### 3) PASOS PARA ESCENARIO 1

Este escenario es similar al base solo que con escalado e imputacion.

#### 3.1) Separar la variable target y dividir el set de datos en train y test.

In [ ]:

# Copia del data set para el escenario 1 (ex Base 2)
df_escBase2 = df.copy()

# Separacion de la variable target
X_escBase2 = df_escBase2.drop('consumo', axis=1)
y_escBase2 = df_escBase2['consumo']

# Dividir el set de datos de entrenamiento y testeo (70-30)
X_train_escBase2, X_test_escBase2, y_train_escBase2, y_test_escBase2 = train_test_split(X_escBase2, y_escBase2, test_size=0.3, random_state=42)

print("Separacion en set de train y test completo usando train_tes_split para escBase2.")
print(f"X_train_escBase2 shape: {X_train_escBase2.shape}")
print(f"X_test_escBase2 shape: {X_test_escBase2.shape}")
print(f"y_train_escBase2 shape: {y_train_escBase2.shape}")
print(f"y_test_escBase2 shape: {y_test_escBase2.shape}")

Separacion en set de train y test completo usando train_tes_split para escBase2.
X_train_escBase2 shape: (53029, 63)
X_test_escBase2 shape: (22728, 63)
y_train_escBase2 shape: (53029,)
y_test_escBase2 shape: (22728,)


#### 3.2) Encoding

In [ ]:
# Identificar las columnas categoricas y numericas para X_train y X_test de escBase2
categorical_cols_escBase2 = X_train_escBase2.select_dtypes(include=['object', 'category']).columns
numerical_cols_escBase2 = X_train_escBase2.select_dtypes(include=np.number).columns

# Encoding mediante get_dummies
X_train_escBase2_encoded = pd.get_dummies(X_train_escBase2, columns=categorical_cols_escBase2, drop_first=True, dtype=int)
X_test_escBase2_encoded = pd.get_dummies(X_test_escBase2, columns=categorical_cols_escBase2, drop_first=True, dtype=int)

print("Encoding completo usando get_dummies para escBase2.")
print(f"X_train_escBase2_encoded shape: {X_train_escBase2_encoded.shape}")
print(f"X_test_escBase2_encoded shape: {X_test_escBase2_encoded.shape}")

Encoding completo usando get_dummies para escBase2.
X_train_escBase2_encoded shape: (53029, 126)
X_test_escBase2_encoded shape: (22728, 126)


#### 3.3) Imputacion

In [ ]:
print("Valores faltantes en X_train_escBase2 (ANTES de la imputacion)")
print(X_train_escBase2_encoded.isnull().sum().sort_values(ascending=False).head(10))

print("\nValores faltantes en X_test_escBase2 (ANTES de la imputacion)")
print(X_test_escBase2_encoded.isnull().sum().sort_values(ascending=False).head(10))


# Inicializacion de SimpleImputer (mean) para escBase2
imputer_escBase2 = SimpleImputer(strategy='mean')

# Columnas numericas despues del encoding para escBase2, se elimina 'id'
numerical_cols_after_encoding_escBase2 = X_train_escBase2_encoded.select_dtypes(include=np.number).columns.tolist()
if 'id' in numerical_cols_after_encoding_escBase2:
    numerical_cols_after_encoding_escBase2.remove('id')


# Imputar X_train_escBase2
X_train_escBase2_encoded[numerical_cols_after_encoding_escBase2] = imputer_escBase2.fit_transform(X_train_escBase2_encoded[numerical_cols_after_encoding_escBase2])

# Imputar X_test_escBase2
X_test_escBase2_encoded[numerical_cols_after_encoding_escBase2] = imputer_escBase2.transform(X_test_escBase2_encoded[numerical_cols_after_encoding_escBase2])


print("\nValores faltantes en X_train_escBase2 (DESPUES de la imputacion)")
print(X_train_escBase2_encoded.isnull().sum().sort_values(ascending=False).head(10))

print("\nValores faltantes en X_test_escBase2 (DESPUES de la imputacion)")
print(X_test_escBase2_encoded.isnull().sum().sort_values(ascending=False).head(10))

Valores faltantes en X_train_escBase2 (ANTES de la imputacion)
dias_con_niebla                      32023
direccion_velocidad_viento_pico      29380
direccion_velocidad_viento_maxima    28847
velocidad_viento_maxima              28847
calificacion_energystar              18832
año_construccion                      1280
enero_temp_promedio                      0
enero_temp_max                           0
febrero_temp_min                         0
febrero_temp_promedio                    0
dtype: int64

Valores faltantes en X_test_escBase2 (ANTES de la imputacion)
dias_con_niebla                      13773
direccion_velocidad_viento_pico      12431
direccion_velocidad_viento_maxima    12235
velocidad_viento_maxima              12235
calificacion_energystar               7877
año_construccion                       557
enero_temp_promedio                      0
enero_temp_max                           0
febrero_temp_min                         0
febrero_temp_promedio                    0
d

#### 3.4) Escalado

In [ ]:
# Identificar columnas numéricas después del encoding e imputación para escBase2
numerical_cols_final_escBase2 = X_train_escBase2_encoded.select_dtypes(include=np.number).columns.tolist()
if 'id' in numerical_cols_final_escBase2:
    numerical_cols_final_escBase2.remove('id') # Eliminar la columna 'id' si existe

# Inicializar StandardScaler para escBase2
scaler_escBase2 = StandardScaler()

# Escalar las columnas numéricas en el conjunto de entrenamiento de escBase2
X_train_escBase2_scaled = scaler_escBase2.fit_transform(X_train_escBase2_encoded[numerical_cols_final_escBase2])

# Escalar las columnas numéricas en el conjunto de prueba de escBase2 (usando el scaler ajustado en el entrenamiento)
X_test_escBase2_scaled = scaler_escBase2.transform(X_test_escBase2_encoded[numerical_cols_final_escBase2])

# Convertir los arrays escalados de vuelta a DataFrames, manteniendo los nombres de las columnas
X_train_escBase2_encoded[numerical_cols_final_escBase2] = X_train_escBase2_scaled
X_test_escBase2_encoded[numerical_cols_final_escBase2] = X_test_escBase2_scaled


print("Escalado completo usando StandardScaler para escBase2.")
print(f"X_train_escBase2_encoded shape después del escalado: {X_train_escBase2_encoded.shape}")
print(f"X_test_escBase2_encoded shape después del escalado: {X_test_escBase2_encoded.shape}")

Escalado completo usando StandardScaler para escBase2.
X_train_escBase2_encoded shape después del escalado: (53029, 126)
X_test_escBase2_encoded shape después del escalado: (22728, 126)


In [ ]:
# Set de datos finales para Escenario Base 2
X_train_final_escBase2 = X_train_escBase2_encoded
X_test_final_escBase2 = X_test_escBase2_encoded
y_train_final_escBase2 = y_train_escBase2
y_test_final_escBase2 = y_test_escBase2

### 4) PASOS PARA ESCENARIO 2

#### 4.1) Planteo de variables a eliminar y creacion de df_esc2

In [ ]:
# Lista de columnas específicas a eliminar
columnas_especificas_a_eliminar = [ 'clase_edificio',
                                    'direccion_velocidad_viento_maxima',
                                    'direccion_velocidad_viento_pico' ,
                                    'velocidad_viento_maxima' ,
                                    'dias_con_niebla']
# Identificar las columnas que contienen '_temp_promedio' en su nombre
columnas_promedio_a_eliminar = [col for col in df.columns if '_temp_promedio' in col]
# Combinar ambas listas de columnas a eliminar
todas_las_columnas_a_eliminar = columnas_especificas_a_eliminar + columnas_promedio_a_eliminar
# Eliminar todas las columnas especificadas en la lista combinada
df_esc2 = df.drop(todas_las_columnas_a_eliminar, axis=1)

#### 4.2) Separar la variable target, dividir set e imputación.

In [ ]:
# Separar la variable target
X_esc2 = df_esc2.drop('consumo', axis=1)
y_esc2 = df_esc2['consumo']

# Dividir el set de datos en train y test
# Uso random_state para que la división sea la misma que en el escenario base
X_train_esc2, X_test_esc2, y_train_esc2, y_test_esc2 = train_test_split(X_esc2, y_esc2, test_size=0.3, random_state=42)

print(f"Dimensiones X_train_esc2 antes de imputar: {X_train_esc2.shape}")
print(f"Dimensiones X_test_esc2 antes de imputar: {X_test_esc2.shape}")

# Columnas específicas a imputar
columnas_a_imputar = ['año_construccion', 'calificacion_energystar']

# Inicializar el imputador (uso la media, probar otra)
imputer_esc2 = SimpleImputer(strategy='mean')

# Aplicar la imputación solo a las columnas especificadas en el conjunto de entrenamiento
X_train_esc2[columnas_a_imputar] = imputer_esc2.fit_transform(X_train_esc2[columnas_a_imputar])

# Aplicar la imputación solo a las columnas especificadas en el conjunto de prueba
X_test_esc2[columnas_a_imputar] = imputer_esc2.transform(X_test_esc2[columnas_a_imputar])

print("\nValores faltantes en X_train_esc2 después de imputar:")
print(X_train_esc2[columnas_a_imputar].isnull().sum())

print("\nValores faltantes en X_test_esc2 después de imputar:")
print(X_test_esc2[columnas_a_imputar].isnull().sum())


Dimensiones X_train_esc2 antes de imputar: (53029, 46)
Dimensiones X_test_esc2 antes de imputar: (22728, 46)

Valores faltantes en X_train_esc2 después de imputar:
año_construccion           0
calificacion_energystar    0
dtype: int64

Valores faltantes en X_test_esc2 después de imputar:
año_construccion           0
calificacion_energystar    0
dtype: int64


#### 4.3) Escalado.

In [ ]:
# Escalar variables numéricas imputadas
# Identificar las columnas numéricas imputadas para escalar (excluir 'id')
# Asegurarnos de que estamos trabajando con los dataframes después de la imputación
numerical_cols_to_scale_esc2 = X_train_esc2.select_dtypes(include=np.number).columns.tolist()
if 'id' in numerical_cols_to_scale_esc2:
    numerical_cols_to_scale_esc2.remove('id')

# Inicializar el escalador
scaler_esc2 = StandardScaler()

# Aplicar fit_transform al set de entrenamiento (imputado)
X_train_escenario2_final = X_train_esc2.copy()
X_train_escenario2_final[numerical_cols_to_scale_esc2] = scaler_esc2.fit_transform(X_train_escenario2_final[numerical_cols_to_scale_esc2])

# Aplicar solo transform al set de testeo (imputado)
X_test_escenario2_final = X_test_esc2.copy()
X_test_escenario2_final[numerical_cols_to_scale_esc2] = scaler_esc2.transform(X_test_escenario2_final[numerical_cols_to_scale_esc2])


print("\nX_train_escenario2_final (primeras 5 filas con numéricas escaladas):")
print(X_train_escenario2_final.head())

print("\nX_test_escenario2_final (primeras 5 filas con numéricas escaladas):")
print(X_test_escenario2_final.head())

# Verificar las estadísticas de las columnas escaladas en train y test
print("\nEstadísticas de columnas numéricas escaladas en X_train_escenario2_final:")
print(X_train_escenario2_final[numerical_cols_to_scale_esc2].describe().loc[['mean', 'std']])

print("\nEstadísticas de columnas numéricas escaladas en X_test_escenario2_final:")
print(X_test_escenario2_final[numerical_cols_to_scale_esc2].describe().loc[['mean', 'std']])

# Set de datos finales para Escenario 2
# X_train_escenario2_final, X_test_escenario2_final, y_train_esc2, y_test_esc2 ya están definidos arriba,
# estas son las variables a usar para el entrenamiento


X_train_escenario2_final (primeras 5 filas con numéricas escaladas):
       factor_año factor_estado           tipo_instalacion  area_edificio  \
13184    1.108561       State_4  Education_Other_classroom      -0.326554   
18163   -1.604780       State_6       Office_Uncategorized       1.691065   
74041    1.108561      State_11        5plus_Unit_Building      -0.394751   
16247   -2.283115       State_6       Office_Uncategorized      11.044662   
50301    0.430226       State_6  Multifamily_Uncategorized      -0.445340   

       año_construccion  calificacion_energystar  elevacion  enero_temp_min  \
13184          0.182698                -0.004180   2.659445       -1.432769   
18163          1.034244                 1.083317  -0.406136        0.164327   
74041          0.732082                -2.266174  -0.216737        1.763341   
16247          0.484859                 0.387319   0.054790       -0.579578   
50301         -0.668849                -0.308679  -0.231822       -0.366

### 5) PASOS PARA ESCENARIO 3

#### 5.1) Eliminacion de variables  

Se listan las variables del data set que no se tendran en cuenta para el escenario 3, justificando cada decision:
'Year_Factor': 'factor_año':
* 'clase_edificio': no aporta informacion adicional, esta contemplada en la variable 'facility_type': 'tipo_instalacion'.
* 'mes_temp_min', 'mes_temp_promedio' y 'mes_temp_max': se resume todo en la variable 'temperatura_promedio'.
*  'direccion_velocidad_viento_maxima', 'direccion_velocidad_viento_pico', 'velocidad_viento_maxima', 'dias_con_niebla': por el gran procentaje de valores faltantes.

El data set queda con las siguientes variables:

    'State_Factor': 'factor_estado',
    'facility_type': 'tipo_instalacion',
    'floor_area': 'area_edificio',
    'year_built': 'año_construccion',
    'energy_star_rating': 'calificacion_energystar',
    'ELEVATION': 'elevacion',
    'cooling_degree_days': 'grados_dia_enfriamiento',
    'heating_degree_days': 'grados_dia_calefaccion',
    'precipitation_inches': 'precipitacion_mm',
    'snowfall_inches': 'nevada_mm',
    'snowdepth_inches': 'profundidad_nieve_mm',
    'avg_temperature': 'temperatura_promedio',
    'days_below_30F':'dias_menos_1C',
    'days_below_20F': 'dias_menos_6C',
    'days_below_10F': 'dias_menos_12C',
    'days_below_0F': 'dias_menos_18C',
    'days_above_80F': 'dias_mas_27C',
    'days_above_90F': 'dias_mas_32C',
    'days_above_100F': 'dias_mas_38C',
    'days_above_110F': 'dias_mas_43C',
    'site_eui': 'consumo',

   




In [ ]:
# Copia del data set para el escenario 3
df_esc3 = df.copy()

In [ ]:
# Columnas a conservar para el escenario 3
cols_to_keep_esc3 = [
    'factor_estado',
    'tipo_instalacion',
    'area_edificio',
    'año_construccion',
    'calificacion_energystar',
    'elevacion',
    'grados_dia_enfriamiento',
    'grados_dia_calefaccion',
    'precipitacion_mm',
    'nevada_mm',
    'profundidad_nieve_mm',
    'temperatura_promedio',
    'dias_menos_1C',
    'dias_menos_6C',
    'dias_menos_12C',
    'dias_menos_18C',
    'dias_mas_27C',
    'dias_mas_32C',
    'dias_mas_38C',
    'dias_mas_43C',
    'consumo',
    'id'
]

# Eliminar columnas no deseadas del dataframe
cols_to_drop_esc3 = [col for col in df_esc3.columns if col not in cols_to_keep_esc3]
df_esc3 = df_esc3.drop(columns=cols_to_drop_esc3)

print("Columnas restantes en df_esc3 para el escenario 3:")
print(df_esc3.columns)
print(f"df_esc3 shape: {df_esc3.shape}")

Columnas restantes en df_esc3 para el escenario 3:
Index(['factor_estado', 'tipo_instalacion', 'area_edificio',
       'año_construccion', 'calificacion_energystar', 'elevacion',
       'grados_dia_enfriamiento', 'grados_dia_calefaccion', 'precipitacion_mm',
       'nevada_mm', 'profundidad_nieve_mm', 'dias_menos_1C', 'dias_menos_6C',
       'dias_menos_12C', 'dias_menos_18C', 'dias_mas_27C', 'dias_mas_32C',
       'dias_mas_38C', 'dias_mas_43C', 'consumo', 'id'],
      dtype='object')
df_esc3 shape: (75757, 21)


#### 5.2) Eliminacion de registros

Se eliminan los registros con algun dato faltante para no imputar e introducir datos estimados en el data set.

In [ ]:
# Calculo de valores faltantes antes de eliminarlos
missing_before = df_esc3.isnull().sum()
print("Valores faltantes por columna ANTES de eliminar registros:")
print(missing_before[missing_before > 0])

print("\nTamaño del set de datos ANTES de eliminar registros:")
print(f"df_esc3 shape: {df_esc3.shape}")

# Numero de registros antes de eliminar
rows_before = df_esc3.shape[0]

# Eliminacion de registros con datos faltantes
df_esc3 = df_esc3.dropna()

# Numero de registros despues de eliminar
rows_after = df_esc3.shape[0]

# Calculo del numero de registros eliminados
rows_removed = rows_before - rows_after
print(f"\nNúmero total de registros eliminados: {rows_removed}")

# Tamaño del data set despues de eliminar registros
print("\nTamaño del set de datos DESPUES de eliminar registros:")
print(f"df_esc3 shape: {df_esc3.shape}")

Valores faltantes por columna ANTES de eliminar registros:
año_construccion            1837
calificacion_energystar    26709
dtype: int64

Tamaño del set de datos ANTES de eliminar registros:
df_esc3 shape: (75757, 21)

Número total de registros eliminados: 27312

Tamaño del set de datos DESPUES de eliminar registros:
df_esc3 shape: (48445, 21)


#### 5.3) Separar la variable target y dividir el set de datos en train y test.

In [ ]:
# Separacion de la variable target
X_esc3 = df_esc3.drop('consumo', axis=1)
y_esc3 = df_esc3['consumo']

# Dividir el set de datos de entrenamiento y testeo (70-30)
X_train_esc3, X_test_esc3, y_train_esc3, y_test_esc3 = train_test_split(X_esc3, y_esc3, test_size=0.3, random_state=10)

print("Separacion en set de train y test completo usando train_tes_split.")
print(f"X_train_esc3 shape: {X_train_esc3.shape}")
print(f"X_test_esc3 shape: {X_test_esc3.shape}")
print(f"y_train_esc3 shape: {y_train_esc3.shape}")
print(f"y_test_esc3 shape: {y_test_esc3.shape}")

Separacion en set de train y test completo usando train_tes_split.
X_train_esc3 shape: (33911, 20)
X_test_esc3 shape: (14534, 20)
y_train_esc3 shape: (33911,)
y_test_esc3 shape: (14534,)


#### 5.4) Encoding

In [ ]:
# Identificar las columnas categoricas y numericas para X_train y X_test
categorical_cols = X_train_esc3.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train_esc3.select_dtypes(include=np.number).columns

# Encoding mediante get_dummies
X_train_esc3_encoded = pd.get_dummies(X_train_esc3, columns=categorical_cols, drop_first=True, dtype=int)
X_test_esc3_encoded = pd.get_dummies(X_test_esc3, columns=categorical_cols, drop_first=True, dtype=int)

print("Encoding completo usando get_dummies.")
print(f"X_train_esc3_encoded shape: {X_train_esc3_encoded.shape}")
print(f"X_test_esc3_encoded shape: {X_test_esc3_encoded.shape}")

Encoding completo usando get_dummies.
X_train_esc3_encoded shape: (33911, 62)
X_test_esc3_encoded shape: (14534, 62)


In [ ]:
# Set de datos finales para Escenario 3
X_train_final_esc3 = X_train_esc3_encoded
X_test_final_esc3 = X_test_esc3_encoded
y_train_final_esc3 = y_train_esc3
y_test_final_esc3 = y_test_esc3

### 6) PASOS PARA ESCENARIO 4


In [ ]:
# Step 0: Copy df1 to df_esc1
df_esc1 = df.copy()

# Step 1: Drop unwanted columns
temp_cols_to_drop = [col for col in df_esc1.columns if ('_temp_min' in col or '_temp_max' in col)]
wind_fog_cols_to_drop = [col for col in df_esc1.columns if ('viento' in col or 'niebla' in col)]

df_esc1 = df_esc1.drop(columns=temp_cols_to_drop + wind_fog_cols_to_drop + ['tipo_instalacion'])

# Step 2: Separate target
X = df_esc1.drop(columns=['consumo'])
y = df_esc1['consumo']

# Step 3: Train/test split before anything else
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 4: Separate column types
numerical_cols = X.select_dtypes(include='number').columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# Step 5: Numerical pipeline (impute + scale)
numerical_pipeline = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=100, random_state=0)),
    ('scaler', StandardScaler())
])

# Fit and transform numeric features
X_train_num = pd.DataFrame(
    numerical_pipeline.fit_transform(X_train[numerical_cols]),
    columns=numerical_cols,
    index=X_train.index
)
X_test_num = pd.DataFrame(
    numerical_pipeline.transform(X_test[numerical_cols]),
    columns=numerical_cols,
    index=X_test.index
)

# Step 6: Categorical preprocessing using get_dummies
X_train_cat = pd.get_dummies(X_train[categorical_cols], drop_first=True, dtype=int)
X_test_cat = pd.get_dummies(X_test[categorical_cols], drop_first=True, dtype=int)

# Align test dummies to match train columns
X_test_cat = X_test_cat.reindex(columns=X_train_cat.columns, fill_value=0)

# Step 7: Concatenate final datasets
X_train_final = pd.concat([X_train_num, X_train_cat], axis=1)
X_test_final = pd.concat([X_test_num, X_test_cat], axis=1)

# Step 8: Renaming of variables

X_train_esc_4=X_train_final
X_test_esc_4=X_test_final
y_train_esc_4=y_train
y_test_esc_4=y_test

# Optional: check the shape
print("X Train shape:", X_train_esc_4.shape)
print("X Test shape:", X_test_esc_4.shape)


X Train shape: (60605, 39)
X Test shape: (15152, 39)


### 7) PASOS PARA ESCENARIO 5

In [ ]:
# Step 0: Copy base
df_esc2 = df.copy()

# Step 1: Drop unwanted columns
drop_cols = [col for col in df_esc2.columns if (
    '_temp_min' in col or '_temp_max' in col or 'viento' in col or 'niebla' in col
)]
df_esc2 = df_esc2.drop(columns=drop_cols)

# Step 2: Create seasonal average columns
seasons = {
    'verano_temp_promedio': ['junio_temp_promedio', 'julio_temp_promedio', 'agosto_temp_promedio'],
    'otoño_temp_promedio': ['septiembre_temp_promedio', 'octubre_temp_promedio', 'noviembre_temp_promedio'],
    'invierno_temp_promedio': ['diciembre_temp_promedio', 'enero_temp_promedio', 'febrero_temp_promedio'],
    'primavera_temp_promedio': ['marzo_temp_promedio', 'abril_temp_promedio', 'mayo_temp_promedio']
}

for season, months in seasons.items():
    df_esc2[season] = df_esc2[months].mean(axis=1)

# Drop monthly columns
monthly_cols = [month for months in seasons.values() for month in months]
df_esc2 = df_esc2.drop(columns=monthly_cols)

# Step 3: Split features and target
X = df_esc2.drop(columns=['consumo'])
y = df_esc2['consumo']

# Step 4: Train-test split (before any transformation)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 5: Identify column types
numerical_cols = X.select_dtypes(include='number').columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# Step 6: Numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Fit numerical pipeline
X_train_num = pd.DataFrame(
    numerical_pipeline.fit_transform(X_train[numerical_cols]),
    columns=numerical_cols,
    index=X_train.index
)
X_test_num = pd.DataFrame(
    numerical_pipeline.transform(X_test[numerical_cols]),
    columns=numerical_cols,
    index=X_test.index
)

# Step 7: Categorical preprocessing with get_dummies
X_train_cat = pd.get_dummies(X_train[categorical_cols], drop_first=True, dtype=int)
X_test_cat = pd.get_dummies(X_test[categorical_cols], drop_first=True, dtype=int)

# Align test set
X_test_cat = X_test_cat.reindex(columns=X_train_cat.columns, fill_value=0)

# Step 8: Concatenate final datasets
X_train_final = pd.concat([X_train_num, X_train_cat], axis=1)
X_test_final = pd.concat([X_test_num, X_test_cat], axis=1)

# Step : Renaming of variables

X_train_esc_5=X_train_final
X_test_esc_5=X_test_final
y_train_esc_5=y_train
y_test_esc_5=y_test

# Optional: check the shape
print("X Train shape:", X_train_esc_5.shape)
print("X Test shape:", X_test_esc_5.shape)

X Train shape: (60605, 90)
X Test shape: (15152, 90)
